In [1]:
import weave
import litellm

In [2]:

# To drop unsupported openai params from the call, set `litellm.drop_params = True`
# Example: ChatGPT O-series models don't support temperature=0.

litellm.drop_params = True

In [3]:
completion = litellm.completion

In [4]:
weave.init("simple_prompting_guide")

Logged in as Weights & Biases user: bikashg.
View Weave data at https://wandb.ai/bikashg-spencer-group/simple_prompting_guide/weave


In [5]:
OPENAI_FAST_MODEL_NAME = "o4-mini"
OPENAI_SMART_MODEL_NAME = "gpt-4.1-mini"

In [44]:
@weave.op()
def get_completion(system_message: str, user_messages: list, model: str, **kwargs):
    # this fromat is specific to openai models only
    formatted_messages = [{"role": "system", "content": system_message}] + user_messages

    # Common arguments for the litellm completion function
    completion_args = {
        "model": model,
        "max_tokens": kwargs.pop('max_tokens', 4096),
        "temperature": kwargs.pop('temperature', 0),
        "messages": formatted_messages
    }
    completion_args.update(kwargs)  # Include any other additional arguments; as such

    # Generate and return the completion
    response = completion(**completion_args)
    return response.json()["choices"][0]["message"]["content"]

In [7]:
question = "Explain the latest prompting techniques and provide an example of each."

### Step 1: Raw Prompting

In [45]:
raw_prompt_response = get_completion(
    system_message= "",
    user_messages = [{"role": "user", "content": question}],
    model = OPENAI_FAST_MODEL_NAME
)

In [46]:
print(raw_prompt_response[0:500] + " ... ")

Here’s a rundown of some of the most effective recent prompting techniques for large language models (LLMs), with a brief definition and a concrete prompt example for each.

1. Zero-Shot Prompting  
  Definition: You give the model a task description (and no examples) and ask it to solve it directly.  
  Example Prompt:  
    “Translate the following sentence into French: ‘The cat sat on the mat.’”  

2. Zero-Shot Chain-of-Thought (CoT)  
  Definition: You ask the model to “think out loud” even  ... 


### Step 2. Prompting with Context

In [10]:
# The context we want to use here is the contents of the prompting guide file
PROMPT_GUIDE = "lilianweng_prompt_engineering.md"

def load_markdown_file(file_path: str) -> str:
    """
    Reads and returns the content of a markdown file specified by its path.

    Parameters:
        file_path (str): The path to the markdown file to be read.

    Returns:
        str: The content of the markdown file as a string.
    """
    with open(file_path, 'r', encoding='utf-8') as file:
        markdown_content = file.read()
    return markdown_content
context = load_markdown_file(PROMPT_GUIDE)

In [11]:
context_prompt_response = get_completion(
    system_message= "",
    user_messages = [{"role": "user", "content": context + "\n\nExplain the latest prompting techniques and provide an example of each"}],
    model = OPENAI_FAST_MODEL_NAME
)

In [12]:
print(context_prompt_response[0:500] + " ... ")

Here is a concise overview of the most popular modern prompting techniques, with a one‐line description of each and a minimal illustrative prompt.

1. Zero-Shot Prompting  
   • Tell the model the task, no examples.  
   • Example (translation):  
     Prompt: “Translate to French: ‘How are you today?’”  
     ⇒ Model: “Comment allez-vous aujourd’hui ?”

2. Few-Shot Prompting  
   • Prepend a handful of (input→output) examples to show style or format.  
   • Example (sentiment classification):   ... 


### Step 3. Condition Responses with a System Prompt

In [13]:
system_message = """
Objective: Simplify prompt engineering concepts for easy understanding. Provide clear examples for each technique.
Tone: Friendly and educational, suitable for beginners.
Context: Assume basic AI knowledge; avoid deep technical jargon.
Guidance: Use metaphors and simple examples to explain concepts. Keep explanations concise and applicable.
Verification: Ensure clarity and relevance in responses, with practical examples.
Benefits: Help users grasp prompt engineering basics, enhancing their AI interaction experience.
"""

In [14]:
system_and_context_prompt_response = get_completion(
    system_message= system_message,
    user_messages = [{"role": "user", "content": context + "\n\nExplain the latest prompting techniques and provide an example of each"}],
    model = OPENAI_FAST_MODEL_NAME
)

In [15]:
print(system_and_context_prompt_response[0:500] + " ... ")

Here’s a friendly, “for-beginners” tour of today’s most popular prompt-engineering tricks. Think of each as a little recipe—first I’ll tell you what it does in plain English (with a quick metaphor), then I’ll show you a super-simple example.  

1. Zero-Shot Prompting  
  • What it is: You walk up to the model cold and ask your question—no examples, no prep.  
  • Metaphor: Like quizzing someone on the spot with “What’s 7 × 8?”  
  • Example:  
    “Translate ‘Good morning’ into French.”  

2. Fe ... 


### Step 4: System Prompts - Inputs. 

### Use f formatted strings to include placeholders in user message. 

In [17]:
user_message_prompt_template = "{input_context}\n\n{input_question}"

In [18]:
# To enable tracking of context and questions dynamically fed at runtime, we will write a function and decorate it with @weave.op
@weave.op()
def format_prompt(template_string: str, **kwargs):
    """
    Formats a prompt template with provided keyword arguments.

    This function takes a template string and a dictionary of keyword arguments,
    then formats the template string using these arguments.

    Parameters:
        prompt_template (str): The template string to be formatted.
        **kwargs (dict): Keyword arguments to format the template string with.

    Returns:
        str: The formatted prompt template.
    """
    return template_string.format(**kwargs)

In [23]:
# Just a simple check
format_prompt(template_string=user_message_prompt_template, input_context="test context", input_question="test question")

'test context\ntest question'

In [22]:
template_instantiated_user_message = format_prompt(template_string=user_message_prompt_template, input_context=context, input_question="""
Explain the differences between zero-shot, few-shot, and chain of thought 
prompting techniques? Please provide a clear explanation and a practical example 
for each technique within a structured format.
""")

In [26]:
system_and_template_instantiated_user_message_response = get_completion(
    system_message= system_message,
    user_messages = [{"role": "user", "content": template_instantiated_user_message}],
    model = OPENAI_FAST_MODEL_NAME
)

In [ ]:
print(system_and_template_instantiated_user_message_response[0:500] + " ... ")

### Step 5: System Prompts - Outputs

In this step, we focus on improving the consistency and structure of our model's outputs by modifying the prompt template. By including specific tags and formatting instructions in the prompt, we can guide the model to respond in a way that is easier to parse and process. Get the ouput in XML, JSON etc. GPT prefers JSON.

In [47]:
system_msg_formatting_instruction = """
Format: Respond within a structured JSON object, using the keys provided in the prompt to organize your response.
Provide a condensed answer under the 'condensed_answer' key, detailed explanations under 'explanation' keys, 
and examples under 'example' keys within each explanation.
"""

user_msg_formatting_instruction = """
You must respond in JSON format.
Your response should follow this structure:
{{ 
  "answer": {{
    "condensed_answer": "CONDENSED_ANSWER",
    "explanation_1": {{
      "detail": "EXPLANATION_1",
      "example": "EXAMPLE_1"
    }},
    "explanation_2": {{
      "detail": "EXPLANATION_2",
      "example": "EXAMPLE_2"
    }},
    ...
  }}
}}
"""

In [48]:
def get_updated_prompt_with_output_formatting_instructions(in_system_message: str, in_user_message: str):
    return in_system_message + "\n" + system_msg_formatting_instruction, in_user_message + "\n" + user_msg_formatting_instruction

In [49]:
output_formatted_system_message, output_formatted_user_message = get_updated_prompt_with_output_formatting_instructions(system_message, template_instantiated_user_message)

In [50]:
output_formatted_system_and_user_message_response = get_completion(
    system_message= output_formatted_system_message,
    user_messages = [{"role": "user", "content": output_formatted_user_message}],
    model = OPENAI_FAST_MODEL_NAME,
    response_format={"type": "json_object"}  # An additional thing here is that we need to specify this.
)

In [51]:
print(output_formatted_system_and_user_message_response[0:500] + " ... ")

{"answer":{"condensed_answer":"Zero-shot asks the model to perform a task with no examples, few-shot provides a handful of demonstrations before the query, and chain-of-thought adds step-by-step reasoning to guide the answer.","explanation_1":{"detail":"Zero-shot prompting is like asking a friend a question without giving any hints or examples. You simply state the task and rely on the model’s existing knowledge.","example":"Prompt: What is the capital of Germany?\\nAnswer:"},"explanation_2":{"d ... 


## Advanced Prompting Techniques

### Zero-shot Prompting

In [43]:
# All the above are examples of zero shot prompting. We didn't show any examples of any output (tone, style, etc.) we would like.

### Few-shot Prompting